In [ ]:
# 필요 라이브러리 설치
'''! pip install pytesseract
! pip install opencv-python-headless
! brew install tesseract-lang 
! pip install matplotlib
! pip install opencv-python
! pip install matplotlib
! pip install torch torchvision torchaudio

! git clone https://github.com/ultralytics/yolov5
! pip install -r /Users/kyungrim/fake-license-plate-detection/yolov5/requirements.txt'''


# 라이브러리 선언
import cv2
import numpy as np
import pandas as pd
import torch
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import os
import torch
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'


# YOLOv5 모델 로드
model_path = './yolov5s.pt'
model = torch.hub.load('./yolov5', 'custom', path=model_path, source='local')

YOLOv5 🚀 v7.0-388-g882c35fc Python-3.12.2 torch-2.5.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
# 함수 선언

# 차량 진입 (차량 이미지 입력)
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8, resize_dim=(800, 600)):
    """
    파일 경로를 읽어 이미지를 로드하고, 크기를 조정합니다.
    :param filename: 이미지 파일 경로
    :param flags: OpenCV 이미지 로드 플래그 (기본값: cv2.IMREAD_COLOR)
    :param dtype: 데이터 타입 (기본값: np.uint8)
    :param resize_dim: 이미지 크기 조정 (너비, 높이)
    :return: 로드된 이미지 또는 None
    """
    try:
        n = np.fromfile(filename, dtype)  # 한글 경로 지원을 위해 np.fromfile 사용
        img = cv2.imdecode(n, flags)     # 이미지 디코딩
        if img is not None and resize_dim:  # 이미지가 로드되었고 크기 조정 설정이 있을 때
            img = cv2.resize(img, resize_dim)
        return img
    except Exception as e:
        print(f"Error reading file {filename}: {e}")
        return None


# 1. 번호판 인식 & 비교

def detecting_car_plate(image) :
    # 이미지 불러오기
    img = cv2.imread(image)

    # 이미지 확인
    if img is None:
        print("차량이 존재하지 않습니다.")
    else:
        print(f"차량이 진입합니다.")

    # YOLOv5로 객체 탐지
    results = model(img)

    detections = results.xyxy[0]  # 탐지된 객체의 좌표
    for i, (*box, conf, cls) in enumerate(detections):  # 좌표, 신뢰도, 클래스
        x1, y1, x2, y2 = map(int, box)  # 바운딩 박스 좌표
        cropped_plate = img[y1:y2, x1:x2]  # 차량번호판 영역 자르기

    return cropped_plate

def extracting_car_number(cropped_plate) :
    config = ('--oem 3 --psm 6')
    text = pytesseract.image_to_string(cropped_plate, lang='kornum+kor', config=config)
    if text is None :
        print("❌ 차량 번호 인식 불가 ❌")
    else :
        return text


# 2. 이미지 매칭 & 비교

def img2hash(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (600, 300))
    avg = gray.mean()
    bi = 1 * (gray > avg)
    return bi

def hamming_distance(a, b):
    a = a.reshape(1,-1)
    b = b.reshape(1,-1)
    # 같은 자리의 값이 서로 다른 것들의 합
    distance = (a !=b).sum()
    return distance

matching_criterion = 200
desc_criterion = 70
def image_match(image, stored_hash):
    # 들어온 차량의 이미지
    income_car_image = image    
    # 차 영상의 해쉬 구하기
    query_hash = img2hash(income_car_image)
    
    # 해당 차량의 저장된 해쉬 불러오기
    stored_hash = stored_hash
    
    # 해밍 거리 산출
    flag = 0
    hamming_dst = hamming_distance(query_hash, stored_hash)
    if hamming_dst/256 < matching_criterion: flag=1; print(f'  ···이미지 매칭량 : {hamming_dst/256}'); print('  ···동일 차량입니다.')
    else: print(f'  ···이미지 매칭량 : {hamming_dst/256}'); print('  ···동일하지 않은 차량입니다.')
        
    return income_car_image, flag



# 3. 특징점 매칭 & 비교

# ORB로 서술자 추출 
detector = cv2.ORB_create()
# BF-Hamming 생성
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)

def get_desc(img):
    img = img
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, desc = detector.detectAndCompute(gray, None)
    return kp, desc

def ORB_knnMatch(income_car_image, flag, stored_kp, stored_desc):
    # 각 영상에 대해 키 포인트와 서술자 추출 
    compare_kp, compare_desc = get_desc(income_car_image)
    # knnMatch, k=2
    matches = matcher.knnMatch(compare_desc, stored_desc, 2)

    # 첫번재 이웃의 거리가 두 번째 이웃 거리의 75% 이내인 것만 추출---⑤
    ratio = 0.75; desc_flag = 0
    good_matches = [first for first,second in matches \
                        if first.distance < second.distance * ratio]
    if len(good_matches) > desc_criterion: desc_flag = 1; print('  ···동일 차량입니다.')
    else: print('  ···동일하지 않은 차량입니다')
    print(f'  ···특징점 매칭량 : {len(good_matches)}/{len(matches)}')
    return desc_flag


# 4. 내부 존재 여부 확인 (데이터베이스 구현 필요)
# 데이터베이스 (아래는 예시)
database = pd.DataFrame({
    '번호판' : ['37허0897', '37머0897', '49하4875'],
    'StoredHash' : [greencar01_hash, greencar02_hash, socar01_hash],
    'KeyPoints' : [greencar01_kp, greencar02_kp, socar01_kp],
    'Descriptors' : [greencar01_desc, greencar02_desc, socar01_desc],
    '내부존재여부' : [0, 0, 0]
})


NameError: name 'np' is not defined

In [ ]:
# main

# 차량 진입
print("진입한 차량의 사진을 업로드 해주세요.")
car = imread(, resize_dim=(800, 600))

# 번호판 인식
cropped_plate = detecting_car_plate(car)
car_number = extracting_car_number(cropped_plate)
if car_number :
    print("진입한 차량의 번호판 : f{car_number}")

if car_number in database['번호판'].tolist() :
    print("통과 1: 정상등록된 차량입니다.")
else :
    print("등록되지 않은 차량입니다.")
    
# 이미지 매칭
income_car_image, flag = image_match(car_image, stored_hash)
if flag == 1:
    print("통과 2 : 이미지 매칭을 통과했습니다.")

# 특징점 매칭
desc_flag = ORB_knnMatch(income_car_image, flag, stored_kp, stored_desc)
if desc_flag == 1:
    print("통과 3 : 특징점 매칭을 통과했습니다.")

# 내부 존재 여부 확인
